In [ ]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [1]:
!pip install opendatasets
!pip install pandas

Библиотеки

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Скачиваем датасет  
Чтение датасета  
Приводим таргет к числовому типу, удаляя строки, которые не удалось конвертировать  
В House price of unit area есть цена 0, что может помешать модели, поэтому удаляем  
Формируем признаки и целевую переменную Таргет - цена Признаки использую все, кроме цены и времени, осталяя только числовые характеристики  
Делюсь на обучающую и тестовую выборки  

In [3]:
od.download("https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression")

pd.set_option("display.max_columns", None)
df = pd.read_csv("house-price-prediction-simplified-for-regression/Real_Estate.csv")

df["House price of unit area"] = pd.to_numeric(df["House price of unit area"], errors="coerce")
df = df.dropna(subset=["House price of unit area"])

df = df[df["House price of unit area"] > 0].copy()

y = df["House price of unit area"]
X = df.drop(columns=["House price of unit area", "Transaction date"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression


100%|██████████| 17.4k/17.4k [00:00<00:00, 57.3MB/s]

Бейзлайн библиотечного леса

Cлучайный лес без тюнинга даёт MAE ≈ 10.44 и RMSE ≈ 11.98, что означает среднюю абсолютную ошибку порядка 10 условных единиц цены за квадратный метр. Коэффициент детерминации R² ≈ 0.21: модель объясняет около 20 % дисперсии целевой переменной, то есть лучше константного предсказания (например, среднего), но запас для улучшения довольно большой

In [ ]:
from sklearn.ensemble import RandomForestRegressor

num_cols = X.columns.tolist()

preprocessor_rf = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

rf_baseline = Pipeline(steps=[
    ("preprocess", preprocessor_rf),
    ("model", RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

rf_baseline.fit(X_train, y_train)
y_pred_base = rf_baseline.predict(X_test)

mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2_base = r2_score(y_test, y_pred_base)

print("Бейзлайн")
print("MAE:", mae_base)
print("RMSE:", rmse_base)
print("R^2:", r2_base)


Бейзлайн
MAE: 10.444865883784049
RMSE: 11.982470064550997
R^2: 0.20616695643691885


1 гипотеза - можно улучшить качество за счёт подбора числа деревьев, глубины, минимального размера листа и числа признаков для сплита

После подбора числа деревьев, глубины, минимального числа объектов в узле и количества признаков при сплите MAE снижается примерно до 9.95, RMSE — до 11.71, а R^2 растёт до ≈ 0.24. Модель начинает чуть лучше приближать реальные цены: ошибка в среднем падает примерно на пол-единицы, а объясняемая доля вариации увеличивается. То есть простая настройка структуры RandomForest (без изменения признаков) уже даёт устойчивое, хоть и не радикальное улучшение качества по сравнению с бейзлайном

In [ ]:
param_grid_rf_1 = {
    "model__n_estimators": [100, 300, 500],
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2"]
}

rf_tuned = Pipeline(steps=[
    ("preprocess", preprocessor_rf),
    ("model", RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

rf_gs_1 = GridSearchCV(
    estimator=rf_tuned,
    param_grid=param_grid_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

rf_gs_1.fit(X_train, y_train)

print("Лучшие параметры:", rf_gs_1.best_params_)
print("Лучший MAE на CV:", -rf_gs_1.best_score_)

best_rf_1 = rf_gs_1.best_estimator_
y_pred_1 = best_rf_1.predict(X_test)

mae_1 = mean_absolute_error(y_test, y_pred_1)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
r2_1 = r2_score(y_test, y_pred_1)

print("\nГипотеза 1")
print("MAE:", mae_1)
print("RMSE:", rmse_1)
print("R^2:", r2_1)


Лучшие параметры: {'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 2, 'model__n_estimators': 300}
Лучший MAE на CV: 10.015323727497934

Гипотеза 1
MAE: 9.951009085203
RMSE: 11.71158196402611
R^2: 0.241653666067977


2 гипотеза - вместо расстояний до станции MRT используем log(1 + distance)

MAE и RMSE остаются на уровне гипотезы 1, R^2 лишь символически увеличивается. Это говорит о том, что для случайного леса, который сам умеет строить сложные нелинейные разбиения по признакам, логарифмирование одной координаты почти не добавляет новой информации

In [ ]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocessor_rf_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

rf_log = Pipeline(steps=[
    ("preprocess", preprocessor_rf_log),
    ("model", RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

rf_gs_2 = GridSearchCV(
    estimator=rf_log,
    param_grid=param_grid_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

rf_gs_2.fit(X_train_log, y_train_log)

print("Лучшие параметры:", rf_gs_2.best_params_)
print("Лучший MAE на CV:", -rf_gs_2.best_score_)

best_rf_2 = rf_gs_2.best_estimator_
y_pred_2 = best_rf_2.predict(X_test_log)

mae_2 = mean_absolute_error(y_test_log, y_pred_2)
rmse_2 = np.sqrt(mean_squared_error(y_test_log, y_pred_2))
r2_2 = r2_score(y_test_log, y_pred_2)

print("\nRandomForestRegressor + log(расстояния до MRT)")
print("MAE:", mae_2)
print("RMSE:", rmse_2)
print("R^2:", r2_2)


Лучшие параметры: {'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 5, 'model__n_estimators': 300}
Лучший MAE на CV: 10.017778655767861

RandomForestRegressor + log(расстояния до MRT)
MAE: 9.947368960535641
RMSE: 11.70524762347358
R^2: 0.24247376448058744


3 гипотеза - логарифм таргета

При переходе к обучению леса на log(price) и обратном преобразовании предсказаний в исходный масштаб MAE снижается до ≈ 9.52, RMSE — до ≈ 11.19, а R^2 заметно растёт до ≈ 0.31. Это означает, что модель лучше справляется с предсказанием как относительно дешёвых, так и дорогих объектов, так как логарифмирование «сжимает» хвост распределения цен и уменьшает влияние экстремальных значений на обучение. В итоге лог-трансформация целевой переменной даёт наибольший прирост качества среди трёх гипотез и явно улучшает баланс между биасом и дисперсией для RandomForestRegressor

In [ ]:
y_log_target = np.log1p(df["House price of unit area"])
X_target = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_target, y_log_target,
    test_size=0.2,
    random_state=42
)

num_cols_t = X_target.columns.tolist()

preprocessor_rf_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols_t),
    ]
)

rf_log_y = Pipeline(steps=[
    ("preprocess", preprocessor_rf_t),
    ("model", RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

rf_gs_3 = GridSearchCV(
    estimator=rf_log_y,
    param_grid=param_grid_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

rf_gs_3.fit(X_train_t, y_train_t)

print("Лучшие параметры:", rf_gs_3.best_params_)
print("Лучший MAE на CV:", -rf_gs_3.best_score_)

best_rf_3 = rf_gs_3.best_estimator_

y_pred_log_t = best_rf_3.predict(X_test_t)
y_pred_t = np.expm1(y_pred_log_t)

y_test_true = np.expm1(y_test_t)

mae_3 = mean_absolute_error(y_test_true, y_pred_t)
rmse_3 = np.sqrt(mean_squared_error(y_test_true, y_pred_t))
r2_3 = r2_score(y_test_true, y_pred_t)

print("\nRandomForestRegressor + логарифм таргета")
print("MAE:", mae_3)
print("RMSE:", rmse_3)
print("R^2:", r2_3)


Лучшие параметры: {'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 500}
Лучший MAE на CV: 0.37656717877557905

RandomForestRegressor + логарифм таргета
MAE: 9.515755249973074
RMSE: 11.190621898727409
R^2: 0.3076193662805524


Собственная реализация

MyDecisionTreeRegressor строит дерево жадно: на каждом узле перебирает признаки и пороги, выбирает разбиение, минимизирующее среднеквадратичную ошибку (MSE), рекурсивно делит выборку, пока не достигнут max_depth / min_samples_split / min_samples_leaf, и в листьях хранит среднее значение таргета
Для каждого дерева бутстрепно выбирает объекты, случайно подвыбирает подмножество признаков (max_features), обучает своё MyDecisionTreeRegressor и сохраняет набор таких базовых моделей  
При предсказании каждое дерево даёт своё прогнозное значение, а финальный ответ случайного леса получается усреднением предсказаний по всем деревьям

In [4]:
class MyDecisionTreeRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.n_features_ = n_features
        self.tree_ = []

        def mse(values):
            if values.size == 0:
                return 0.0
            mean = values.mean()
            return np.mean((values - mean) ** 2)

        def best_split(X_node, y_node, depth):
            n_samples_node, n_features_node = X_node.shape
            if n_samples_node < self.min_samples_split:
                return None, None, None, None
            if self.max_depth is not None and depth >= self.max_depth:
                return None, None, None, None

            best_feature = None
            best_threshold = None
            best_score = np.inf
            best_left_idx = None
            best_right_idx = None

            for feature in range(n_features_node):
                values = X_node[:, feature]
                thresholds = np.unique(values)
                for thr in thresholds:
                    left_idx = values <= thr
                    right_idx = values > thr
                    if left_idx.sum() < self.min_samples_leaf or right_idx.sum() < self.min_samples_leaf:
                        continue
                    mse_left = mse(y_node[left_idx])
                    mse_right = mse(y_node[right_idx])
                    score = (left_idx.sum() * mse_left + right_idx.sum() * mse_right) / n_samples_node
                    if score < best_score:
                        best_score = score
                        best_feature = feature
                        best_threshold = thr
                        best_left_idx = left_idx
                        best_right_idx = right_idx

            if best_feature is None:
                return None, None, None, None
            return best_feature, best_threshold, best_left_idx, best_right_idx

        def build_tree(X_node, y_node, depth):
            node = {}
            node["value"] = float(y_node.mean())

            feature, threshold, left_idx, right_idx = best_split(X_node, y_node, depth)
            if feature is None:
                node["feature"] = None
                node["threshold"] = None
                node["left"] = None
                node["right"] = None
                return node

            node["feature"] = feature
            node["threshold"] = threshold
            node["left"] = build_tree(X_node[left_idx], y_node[left_idx], depth + 1)
            node["right"] = build_tree(X_node[right_idx], y_node[right_idx], depth + 1)
            return node

        self.tree_ = build_tree(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        while node["feature"] is not None:
            if x[node["feature"]] <= node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["value"]

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        y_pred = np.zeros(n_samples, dtype=float)
        for i in range(n_samples):
            y_pred[i] = self._predict_one(X[i], self.tree_)
        return y_pred


class MyRandomForestRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.random_state = random_state

    def _get_n_features_to_sample(self, n_features):
        if isinstance(self.max_features, int):
            return min(self.max_features, n_features)
        if isinstance(self.max_features, float):
            return max(1, int(self.max_features * n_features))
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        return n_features

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.trees_ = []
        self.features_subsets_ = []

        n_features_sub = self._get_n_features_to_sample(n_features)

        for i in range(self.n_estimators):
            if self.bootstrap:
                indices = rng.randint(0, n_samples, size=n_samples)
            else:
                indices = np.arange(n_samples)

            feat_indices = rng.choice(n_features, size=n_features_sub, replace=False)

            X_boot = X[indices][:, feat_indices]
            y_boot = y[indices]

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=None
            )
            tree.fit(X_boot, y_boot)

            self.trees_.append(tree)
            self.features_subsets_.append(feat_indices)

        return self

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        preds = np.zeros((n_samples, len(self.trees_)), dtype=float)

        for i, (tree, feat_idx) in enumerate(zip(self.trees_, self.features_subsets_)):
            preds[:, i] = tree.predict(X[:, feat_idx])

        return preds.mean(axis=1)


Бейзлайн

MAE ≈ 10.09 — в среднем модель ошибается примерно на 10 единиц цены за квадратный метр  
RMSE ≈ 12.46 — квадратичная ошибка чуть выше MAE, что говорит о наличии некоторых сильно «выбивающихся» объектов, но без экстремального влияния  
R^2 ≈ 0.14 — модель объясняет около 14 % дисперсии таргета, то есть качество пока довольно скромное, но заметно лучше, чем у одиночного решающего дерева, которое сильно переобучалось и давало отрицательный R^2


In [ ]:
num_cols = X.columns.tolist()

preprocessor_my_rf = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

my_rf_baseline = Pipeline(steps=[
    ("preprocess", preprocessor_my_rf),
    ("model", MyRandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        random_state=42
    ))
])

my_rf_baseline.fit(X_train, y_train)
y_pred_my_base = my_rf_baseline.predict(X_test)

mae_my_base = mean_absolute_error(y_test, y_pred_my_base)
rmse_my_base = np.sqrt(mean_squared_error(y_test, y_pred_my_base))
r2_my_base = r2_score(y_test, y_pred_my_base)

print("Бейзлайн")
print("MAE:", mae_my_base)
print("RMSE:", rmse_my_base)
print("R^2:", r2_my_base)


Бейзлайн
MAE: 10.090213531174243
RMSE: 12.455470340974834
R^2: 0.1422578915277779


1 гипотеза

Лучшие параметры дают совсем небольшой прирост по R^2 (с ≈0.14 до ≈0.15) и чуть более низкий MAE, то есть тюнинг помогает, но эффект очень умеренный из-за ограниченности данных

In [7]:
num_cols = X.columns.tolist()

preprocessor_my_rf = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
    ]
)

param_grid_my_rf_1 = {
    "model__n_estimators": [100, 300, 500],
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2"]
}

my_rf_tuned_1 = Pipeline(steps=[
    ("preprocess", preprocessor_my_rf),
    ("model", MyRandomForestRegressor(
        bootstrap=True,
        random_state=42
    ))
])

my_rf_gs_1 = GridSearchCV(
    estimator=my_rf_tuned_1,
    param_grid=param_grid_my_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_rf_gs_1.fit(X_train, y_train)

print("Лучшие параметры:", my_rf_gs_1.best_params_)
print("Лучший MAE на CV:", -my_rf_gs_1.best_score_)

best_my_rf_1 = my_rf_gs_1.best_estimator_
y_pred_my_1 = best_my_rf_1.predict(X_test)

mae_my_1 = mean_absolute_error(y_test, y_pred_my_1)
rmse_my_1 = np.sqrt(mean_squared_error(y_test, y_pred_my_1))
r2_my_1 = r2_score(y_test, y_pred_my_1)

print("\nГипотеза 1")
print("MAE:", mae_my_1)
print("RMSE:", rmse_my_1)
print("R^2:", r2_my_1)


Лучшие параметры: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 10, 'model__n_estimators': 100}
Лучший MAE на CV: 10.323365456032814

Гипотеза 1
MAE: 10.061651862392814
RMSE: 12.366973051530215
R^2: 0.15440326755218836


2 гипотеза

Качество чуть лучше бейзлайна и сопоставимо с гипотезой 1, но логарифмирование расстояния само по себе даёт только небольшой прирост по R^2 и MAE

In [8]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

preprocessor_my_rf_log = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_log),
    ]
)

my_rf_tuned_2 = Pipeline(steps=[
    ("preprocess", preprocessor_my_rf_log),
    ("model", MyRandomForestRegressor(
        bootstrap=True,
        random_state=42
    ))
])

my_rf_gs_2 = GridSearchCV(
    estimator=my_rf_tuned_2,
    param_grid=param_grid_my_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_rf_gs_2.fit(X_train_log, y_train_log)

print("Лучшие параметры:", my_rf_gs_2.best_params_)
print("Лучший MAE на CV:", -my_rf_gs_2.best_score_)

best_my_rf_2 = my_rf_gs_2.best_estimator_
y_pred_my_2 = best_my_rf_2.predict(X_test_log)

mae_my_2 = mean_absolute_error(y_test_log, y_pred_my_2)
rmse_my_2 = np.sqrt(mean_squared_error(y_test_log, y_pred_my_2))
r2_my_2 = r2_score(y_test_log, y_pred_my_2)

print("\nMyRandomForestRegressor + log(расстояния до MRT)")
print("MAE:", mae_my_2)
print("RMSE:", rmse_my_2)
print("R^2:", r2_my_2)


Лучшие параметры: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 10, 'model__n_estimators': 100}
Лучший MAE на CV: 10.323365456032814

MyRandomForestRegressor + log(расстояния до MRT)
MAE: 10.061651862392814
RMSE: 12.366973051530215
R^2: 0.15440326755218836


3 гипотеза

В результате MAE и RMSE заметно уменьшаются, а R^2 растёт до ~0.256 — это лучшая конфигурация для моего леса по сравнению с бейзлайном и предыдущими гипотезами

In [9]:
y_log_target = np.log1p(df["House price of unit area"])
X_target = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_target, y_log_target,
    test_size=0.2,
    random_state=42
)

num_cols_t = X_target.columns.tolist()

preprocessor_my_rf_t = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols_t),
    ]
)

my_rf_tuned_3 = Pipeline(steps=[
    ("preprocess", preprocessor_my_rf_t),
    ("model", MyRandomForestRegressor(
        bootstrap=True,
        random_state=42
    ))
])

my_rf_gs_3 = GridSearchCV(
    estimator=my_rf_tuned_3,
    param_grid=param_grid_my_rf_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_rf_gs_3.fit(X_train_t, y_train_t)

print("Лучшие параметры:", my_rf_gs_3.best_params_)
print("Лучший MAE на CV:", -my_rf_gs_3.best_score_)

best_my_rf_3 = my_rf_gs_3.best_estimator_

y_pred_log_t = best_my_rf_3.predict(X_test_t)
y_pred_t = np.expm1(y_pred_log_t)

y_test_true = np.expm1(y_test_t)

mae_my_3 = mean_absolute_error(y_test_true, y_pred_t)
rmse_my_3 = np.sqrt(mean_squared_error(y_test_true, y_pred_t))
r2_my_3 = r2_score(y_test_true, y_pred_t)

print("\nMyRandomForestRegressor + логарифм таргета")
print("MAE:", mae_my_3)
print("RMSE:", rmse_my_3)
print("R^2:", r2_my_3)

Лучшие параметры: {'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Лучший MAE на CV: 0.3856605445046651

MyRandomForestRegressor + логарифм таргета
MAE: 9.687211893877322
RMSE: 11.60197299478937
R^2: 0.2557820036281576


В задаче регрессии по ценам жилья случайный лес ожидаемо даёт устойчивое качество: библиотечный RandomForestRegressor после подбора гиперпараметров и логарифмирования таргета выходит примерно на MAE ≈ 9.5 и R^2 ≈ 0.31, моя реализация леса показывает похожие тенденции, но остаётся немного слабее (MAE ≈ 9.7, R^2 ≈ 0.26). Во всех вариантах простой тюнинг гиперпараметров даёт умеренный прирост качества, логарифмирование расстояния до MRT почти не меняет картину, а логарифмирование таргета даёт самый заметный выигрыш по метрикам. В целом можно сказать, что и библиотечный, и собственный лес хорошо улавливают нелинейные зависимости в данных, а моя реализация ведёт себя адекватно, но ожидаемо чуть проигрывает оптимизированной реализации sklearn по точности